# Philippine Junior Data Science Challenge 2

**TEAM MATHEMARIZZ**
* MARIANO, Isaiah John L.
* MONTEALTO, Meluisa D.
* REGALARIO, Jeremiah Daniel A.


(1) Segmentation of Core Mass clients from far-flung areas based on client information and behavior
from April to June 2023; and<br>
(2) Market and product recommendations, and next best actions per created segment. <br><br>
For this challenge, the goal of our segmentation process is to be able to create products that would not only entice the masses into signing up to become BPI clients, but to also cater to undeserved populations through financial inclusion and to uphold BPI's vision to build a better Philippines.

# Running

Set up the running option for this script.

In [ ]:
run = {
    0: 'dataset', # local run
    1: '/content' # Google Colaboratory
}

# Select an option to run by setting variable k by 0 or 1
k = 1

address = run[k]

# Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

try:
    from kneed import KneeLocator
except:
    !pip install kneed
    from kneed import KneeLocator


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Data Cleaning



## Customer Info

### Extracting Data


In [ ]:
cust_info = pd.read_excel(f'{address}/CUSTOMER INFO_PJDSC.xlsx')
cust_info.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/CUSTOMER INFO_PJDSC.xlsx'

In [ ]:
cust_info.info()

In [ ]:
cust_info.describe(include='all')

In [ ]:
cust_info_row, cust_info_col = cust_info.shape
print(f"Number of Rows: {cust_info_row}")
print(f"Number of Columns: {cust_info_col}")

### Remove Duplicates

In [ ]:
cust_info_duplicates = cust_info.duplicated().sum()
print(f"Number of duplicates: {cust_info_duplicates}")

In [ ]:
cust_info.drop_duplicates(inplace=True)

In [ ]:
cust_info.info()

### Check Unique Values

In [ ]:
dataset = cust_info

for column in dataset.columns:
  unique_count = dataset[column].nunique()
  unique_values = dataset[column].unique()

  print(f'''
{column}
  {unique_values}
Length: {unique_count}
''')

### Null Values Detection

In [ ]:
cust_info.isna().sum()

In [ ]:
cust_info.isna().sum().plot(kind='bar')

The `AGE` column has null values, but there are columns with "NO_DATA".

In [ ]:
cust_info_nulls = cust_info.replace('NO_DATA', np.nan, inplace=False).isna().sum()
cust_info_nulls

In [ ]:
cust_info.replace('NO_DATA', np.nan, inplace=False).isna().sum().plot(kind='bar')

In [ ]:
total_cust_info = len(cust_info)
(cust_info_nulls / total_cust_info) * 100

### Remove Unnecessary Columns

In [ ]:
cust_info.nunique()

Remove `IDV_OR_ORG`, `SUBSEGMENT`, and `SEGMENT` columns.

In [ ]:
cust_info.drop(columns=['IDV_OR_ORG', 'SEGMENT', 'SUBSEGMENT'], inplace=True)

In [ ]:
cust_info.info()

### Outliers Detection

In [ ]:
num_col = ['AGE', 'TENURE']

plt.figure(figsize=(30, 30))
for i, c in enumerate(num_col):
  plt.subplot(6, 4, i+1)
  sns.boxplot(data=cust_info, x=cust_info[c])

In [ ]:
def perc_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    num = df[(df[column] < lower_bound) | (df[column] > upper_bound)].shape[0]
    perc = (num / df.shape[0]) * 100
    return perc

In [ ]:
age_outliers = perc_outliers(cust_info, 'AGE')
tenure_outliers = perc_outliers(cust_info, 'TENURE')
print(f'Percentage of outliers for AGE: {(age_outliers):.2f}%')
print(f'Percentage of outliers for TENURE: {(tenure_outliers):.2f}%')

In [ ]:
cust_info.head()

### Questions to be Answered

#### Sultan Kudarat Core Mass Clients (1)

*How many Core Mass clients from Sultan Kudarat are in the Customer Info dataset?*

In [ ]:
ans1 = cust_info.loc[cust_info['PROVINCE'] == 'SULTAN KUDARAT']
print(f"Number of Sultan Kudarat Core Mass Clients: {len(ans1)}")

**Answer: There are 154 Core Mass clients from Sultan Kudarat that are in the Customer Info dataset.**

#### Null Values (5)

*For each dataset, which columns contain null values?*

Answer: For the customer info dataset,
- It can be determined that `AGE` is the only one with null values.
- Furthermore, assuming that `NO_DATA` represents null values, then the columns with null values include `EDUCATION`, `DIGITAL_FLAG`, and `INCOME_SOURCE`, aside from `AGE`. <br>
- However, `NO_DATA` may represent a legitimate data point.
- For `DIGITAL_FLAG`, the person may fall in between being tech savvy and not tech savvy.
- For `INCOME_SOURCE`, the person may not have a source of income at all. For `EDUCATION`, `NO_DATA` seems to be a legit null value since having no formal education falls under `LOW`.

## Credit Card Transactions

### Extracting Data

In [ ]:
credit_trans = pd.read_excel(f'{address}/CREDIT CARD TXNS_PJDSC.xlsx')
credit_trans.head()

In [ ]:
credit_trans.info()

In [ ]:
credit_trans.describe(include='all')

In [ ]:
credit_trans_row, credit_trans_col = credit_trans.shape
print(f"Number of Rows: {credit_trans_row}")
print(f"Number of Columns: {credit_trans_col}")

### Remove Duplicates

In [ ]:
credit_trans_duplicates = credit_trans.duplicated().sum()
print(f"Number of duplicates: {credit_trans_duplicates}")

### Check Unique Values

In [ ]:
dataset = credit_trans

for column in dataset.columns:
  unique_count = dataset[column].nunique()
  unique_values = dataset[column].unique()

  print(f'''
{column}
  {unique_values}
Length: {unique_count}
''')

### Null Values Detection

In [ ]:
credit_trans_nulls = credit_trans.isna().sum()
credit_trans_nulls

In [ ]:
credit_trans.isna().sum().plot(kind='bar')

In [ ]:
total_credit_trans = len(credit_trans)
(credit_trans_nulls / total_credit_trans) * 100

### Remove Unnecessary Columns

In [ ]:
credit_trans.nunique()

Since `MERCH_NAME` does not give much information, remove this column.

In [ ]:
credit_trans.drop(columns=['MERCH_NAME'], inplace=True)

In [ ]:
credit_trans.info()

### Outliers Detection

In [ ]:
sns.boxplot(data=credit_trans, x=credit_trans['AMOUNT'])

In [ ]:
credit_trans_outliers = perc_outliers(credit_trans, 'AMOUNT')
print(f'Percentage of outliers for AMOUNT: {credit_trans_outliers:.2f}%')

In [ ]:
credit_trans.head()

### Rename Columns

In [ ]:
suffix = "_cred"

col = ['MERCH_GROUP', 'DATE', 'AMOUNT', 'CHANNEL', 'COUNTRY']
columns_to_rename = {s: f"{s}{suffix}" for s in col}

credit_trans = credit_trans.rename(columns=columns_to_rename)
credit_trans.head()

### Questions to be Answered

#### Top Three Countries (4)

*What are the top three countries wherein Core Mass clients performed credit card transactions most frequently?*

In [ ]:
credit_trans['COUNTRY_cred'].value_counts().sort_values().plot(kind='barh')

**Answer: The top three countries wherein Core Mass clients performed credit card transactions most frequently are PH, Country 2, and Country 4, respectively.**

#### Null Values (5)

*For each dataset, which columns contain null values?*

Answer: For the credit card transactions dataset, the `MERCH_GROUP` and `COUNTRY` columns have null values.

## Debit Card Transactions

### Extraction

In [ ]:
debit_trans = pd.read_excel(f'{address}/DEBIT CARD TXNS_PJDSC.xlsx')
debit_trans.head()

In [ ]:
debit_trans.info()

In [ ]:
debit_trans.describe(include='all')

In [ ]:
debit_trans_row, debit_trans_col = debit_trans.shape
print(f'''Number of Rows: {debit_trans_row}
Number of Columns: {debit_trans_col}''')

### Remove Duplicates

In [ ]:
debit_trans_duplicates = debit_trans.duplicated().sum()
print(f"Number of duplicates: {debit_trans_duplicates}")

### Check Unique Values

In [ ]:
dataset = debit_trans

for column in dataset.columns:
  unique_count = dataset[column].nunique()
  unique_values = dataset[column].unique()

  print(f'''
{column}
  {unique_values}
Length: {unique_count}
''')

### Remove March 31 Transactions

In [ ]:
debit_trans = debit_trans[debit_trans['DATE'].dt.month >= 4]
debit_trans.describe(include='all')

### Null Values Detection

In [ ]:
debit_trans_nulls = debit_trans.isna().sum()
debit_trans_nulls

### Remove Unnecessary Columns

The time of the day of the transaction is of no interest. Remove the `TIME` column.

In [ ]:
debit_trans.drop(columns=['TIME'], inplace=True)

In [ ]:
debit_trans.info()

### Outliers Detection

In [ ]:
sns.boxplot(data=debit_trans, x=debit_trans['AMOUNT'])

In [ ]:
debit_trans_outliers = perc_outliers(debit_trans, 'AMOUNT')
print(f'Percentage of outliers for AMOUNT: {debit_trans_outliers:.2f}%')

### Rename Columns

In [ ]:
suffix = "_deb"

col = ['DATE', 'AMOUNT', 'CHANNEL']
columns_to_rename = {s: f"{s}{suffix}" for s in col}

debit_trans = debit_trans.rename(columns=columns_to_rename)
debit_trans.head()

### Questions to be Answered

#### Null Values (5)

*For each dataset, which columns contain null values?*

**Answer: For the debit card transactions dataset, there are no columns that contain null values.**

## Financial Digital Transactions

### Extraction

In [ ]:
fin_trans = pd.read_excel(f'{address}/FINANCIAL DIGITAL TXNS_PJDSC.xlsx')
fin_trans.head()

In [ ]:
fin_trans.info()

In [ ]:
fin_trans.describe(include='all')

In [ ]:
fin_trans_row, fin_trans_col = fin_trans.shape
print(f'''Number of Rows: {fin_trans_row}
Number of Columns: {fin_trans_col}''')

### Remove Duplicates

In [ ]:
print(f"Number of duplicates: {fin_trans.duplicated().sum()}")

### Check Unique Values

In [ ]:
dataset = fin_trans

for column in dataset.columns:
  unique_count = dataset[column].nunique()
  unique_values = dataset[column].unique()

  print(f'''
{column}
  {unique_values}
Length: {unique_count}
''')

### Null Values Detection

In [ ]:
fin_trans_nulls = fin_trans.isna().sum()
fin_trans_nulls

### Outliers Detection

In [ ]:
sns.boxplot(data=fin_trans, x=fin_trans['AMOUNT'])

In [ ]:
print('Percentage of outliers for AMOUNT:', perc_outliers(fin_trans, 'AMOUNT'))

### Date Only

In [ ]:
fin_trans['DATE'] = fin_trans['DATE_AND_TIME'].dt.normalize()
fin_trans.drop(columns=['DATE_AND_TIME'], inplace=True)
fin_trans.head()

### Rename Columns

In [ ]:
suffix = "_fin"

col = ['DATE', 'AMOUNT', 'CHANNEL', 'TRAN_TYPE']
columns_to_rename = {s: f"{s}{suffix}" for s in col}

fin_trans = fin_trans.rename(columns=columns_to_rename)
fin_trans.head()

### Questions to be Answered

#### Total Financial Digital Transaction Amount in May 2023 (2)

*In May 2023, what was the total financial digital transaction amount produced by Core Mass clients?*

In [ ]:
may_only = fin_trans[fin_trans['DATE_fin'].dt.month == 5]
ans2 = may_only['AMOUNT_fin'].sum()
print(f"In May 2023, the total financial digital transaction amount produced by Core Mass clients was: {ans2}")

**Answer: In May 2023, the total financial digital transaction amount produced by Core Mass clients was P 210,094,389.85**

#### Null Values (5)

*For each dataset, which columns contain null values?*

**Answer: For the digital transactions dataset, there is no column that contains null values.**

## Incoming Instapay Transactions

### Extraction

In [ ]:
inc_inst = pd.read_excel(f'{address}/INCOMING INSTAPAY TXNS_PJDSC.xlsx')
inc_inst.head()

In [ ]:
inc_inst.info()

In [ ]:
inc_inst.describe(include='all')

In [ ]:
inc_inst_row, inc_inst_col = inc_inst.shape
print(f'''Number of Rows: {inc_inst_row}
Number of Columns: {inc_inst_col}''')

### Remove Duplicates

In [ ]:
print(f"Number of duplicates: {inc_inst.duplicated().sum()}")

### Check Unique Values

In [ ]:
for col in inc_inst.columns:
  print(col)
  print(inc_inst[col].unique())
  print("Length:", inc_inst[col].nunique())
  print('\n')

### Null Values Detection

In [ ]:
inc_inst_nulls = inc_inst.isna().sum()
inc_inst_nulls

### Remove Unnecessary Columns

The time of the day of the transaction is of no interest. Remove the `TIME` column.

In [ ]:
inc_inst.drop(columns=['TIME'], inplace=True)

In [ ]:
inc_inst.info()

### Outliers Detection

In [ ]:
sns.boxplot(data=inc_inst, x=inc_inst['AMOUNT'])

In [ ]:
inc_inst_outliers = perc_outliers(inc_inst, 'AMOUNT')
print(f'Percentage of outliers for AMOUNT: {inc_inst_outliers}')

### Rename Columns

In [ ]:
suffix = "_inc_inst"

col = ['DATE', 'AMOUNT', 'BPI_ACCT_TYPE', 'SRC_BANK_NAME']
columns_to_rename = {s: f"{s}{suffix}" for s in col}

inc_inst = inc_inst.rename(columns=columns_to_rename)
inc_inst.head()

### Questions to be Answered

#### Null Values (5)

*For each dataset, which columns contain null values?*

**Answer: For the incoming INSTAPAY transactions dataset, there is no column that contains null values.**

## Outgoing Instapay Transactions

### Extraction

In [ ]:
out_inst = pd.read_excel(f'{address}/OUTGOING INSTAPAY TXNS_PJDSC.xlsx')
out_inst.head()

In [ ]:
out_inst.info()

In [ ]:
out_inst.describe(include='all')

In [ ]:
out_inst_row, out_inst_col = out_inst.shape
print(f'''Number of Rows: {out_inst_row}
Number of Columns: {out_inst_col}''')

### Remove Duplicates

In [ ]:
print(f"Number of duplicates: {out_inst.duplicated().sum()}")

### Check Unique Values

In [ ]:
dataset = out_inst

for column in dataset.columns:
  unique_count = dataset[column].nunique()
  unique_values = dataset[column].unique()

  print(f'''
{column}
  {unique_values}
Length: {unique_count}
''')

### Null Values Detection

In [ ]:
out_inst_nulls = out_inst.isna().sum()
out_inst_nulls

### Remove Unnecessary Columns

The time of the day of the transaction is of no interest. Remove the `TIME` column.

In [ ]:
out_inst.drop(columns=['TIME'], inplace=True)

In [ ]:
out_inst.info()

### Outliers Detection

In [ ]:
sns.boxplot(data=out_inst, x=out_inst['AMOUNT'])

In [ ]:
out_inst_outliers = perc_outliers(out_inst, 'AMOUNT')
print(f'Percentage of outliers for AMOUNT: {out_inst_outliers}')

### Rename Columns

In [ ]:
suffix = "_out_inst"

col = ['DATE', 'AMOUNT', 'CHANNEL', 'RCV_BANK_NAME']
columns_to_rename = {s: f"{s}{suffix}" for s in col}

out_inst = out_inst.rename(columns=columns_to_rename)
out_inst.head()

### Questions to be Answered

#### Bank that was most frequently treated as recipient for Outgoing Instapay Transactions (3)

*What bank was most frequently treated as recipient by Core Mass for their outgoing Instapay transactions?*

In [ ]:
out_inst['RCV_BANK_NAME_out_inst'].value_counts().plot(kind='bar', fontsize=8)

**Answer: The bank that was most frequently treated as recipient by Core Mass for their outgoing Instapay transactions was Human.**

#### Null Values (5)

*For each dataset, which columns contain null values?*

**Answer: For the outgoing INSTAPAY transactions dataset, there is no column that contains null values.**

## Handling Null Values and Outliers

Since `EDUCATION` and `DIGITAL_FLAG` have many missing values, we have decided that it is best to impute rather than drop. Random Forest Classifier can be used for these features. Missing values in `AGE` and `INCOME_SOURCE` can be dropped since they only constitute a small percentage of the dataset.

Outliers would be removed if it would help better illustrate the distributions of some features.

# Exploratory Data Analysis

## Univariate Analysis

### Customer Information

In this section, we analyzed the customer's demographics based on the `CUSTOMER_INFO` dataset given. This is where we lay the foundations of our exploratory data analysis.

In [ ]:
plt.style.use('seaborn-v0_8')
cust_info_cat_bar = ['PROVINCE', 'REGION', 'INCOME_SOURCE']
plt.figure(figsize=(10, 40))

for i, c in enumerate(cust_info_cat_bar):
  plt.subplot(3, 1, i+1)
  cust_info[c].value_counts().sort_values(ascending=True).plot(
      kind='barh',
      title=f"Bar Graph of Frequency of {c}",
      ylabel=c,
      xlabel='Frequency',
      fontsize=8
  )

plt.style.use('fast')

It is not worth exploring the relationship of geographic location and other features since the products that we would propose would not look at location, but more on the personal side of the demographics and spending habits.

Clients who have a salary represent a huge chunk of the dataset.

In [ ]:
cust_info_cat_pie = ['EDUCATION', 'MARITAL_STATUS', 'BUSINESS_OWNER', 'DIGITAL_FLAG','GENDER']

plt.figure(figsize=(20, 20))

for i, c in enumerate(cust_info_cat_pie):
  cat = c.title().replace('_', ' ')
  plt.subplot(5, 3, i+1)
  plt.style.use('fast')

  sorted = cust_info[c].value_counts().sort_values(ascending=False)
  sorted.plot(kind='pie', fontsize=9, autopct='%1.f%%', startangle=-270, labels=None, counterclock=False)
  plt.legend(labels=sorted.rename(lambda x: x.title().replace('_', ' ')).index, bbox_to_anchor=(1, 1))
  plt.title(f"Proportion of Customers' {cat + 'ship' if c == 'BUSINESS_OWNER' else cat}", weight="bold")
  plt.tight_layout()
  plt.ylabel('')

**Observations:**
- Based on the pie chart, it can be deduced that majority of the customers have **"Mid"** (Associate or Bachelor's Degree) level of education, comprising 46% of the customers. Meanwhile, it is followed by customers with no data about their education, accounting for 28.7% of the customers. On the other hand, 23.8% of the customers have **"Low"** (no formal education, elementary or high school gradu) level of education. And lastly, 2% of them have **"High"** (master's or doctorate degree).
- From the graph, it can be deduced that the primary income source of the customers are from their **salary**, and is greater than all the other income sources combined. It is followed by allowance, pension, remittance, business and etc.
- Most of the customers **do not own business** (95%), while only 5% of them own business.
- Almost half of the customers are **traditional** (47%), while almost a third are **tech savvy** (30%). There are no data available for the remaining customers (23%)
- There are more **female** customers (58%) compared to the number of male customers (40%). There are also non-binary customers that comprises the 2% of the population.  

**Interpretations:**
- Mid Education, not being a business owner, and traditional digital flag being prevalent suggests that the clients from far-flung areas are considered to be **"average citizens"**. Even if the Philippines is shifting to a society that considerably relies on technology, with ~144.5% of the population are active cellular mobile connections [(source)](https://datareportal.com/reports/digital-2023-philippines), there is still a significant portion of the people that are not very digital savvy. Furthermore, it is not worth exploring the relationship of being or not being a business owner to other features since almost all clients are not business owners.
- 'Single' has the highest rate of bank ownership, as single individuals often focus on their work, saving for their future, and future adventures. 'Married' individuals also have a significant number of bank accounts since they are preparing for their children's future and maintaining joint accounts with their partners. 'Widowed' individuals have accumulated only 3% of bank ownership, possibly because they are older and may not be familiar with the process of opening an account. Alternatively, some may entrust their money to relatives, and it's also likely that many have inherited substantial sums of money, making the need for interest from banks unnecessary.

In [ ]:
cust_info_cat_hist = {'AGE':0, 'TENURE':1}
plt.figure(figsize=(30, 30))
for i,c in enumerate(cust_info_cat_hist):
  plt.subplot(4,3, i+1)
  cust_info[c].plot(kind='hist', title=f"Histogram of {c}", xlabel=c)

**Observations:**
- From the histogram of age, we get that most of the customers come from the **25-37** age bracket. It is then followed by 38-50, 13-25, 50-62, 62-74, 74-86, and 86-98.
- The tenure of most of the customers ranges from **0-3 years** comprised by ~1500 customers.  It is followed closely by 3-6, 6-9, 9-12, 12-15, 15-18, 21-24, 18-21 and lastly, 24-27 years.

**Interpretations:**
- The distribution of `AGE` appears to be normal. Many of the clients fall under the **Millenial generation** (~25-40 years old). They are most likely workers who are earning through salary due to the sheer amount of representation salaried clients have in the dataset.
- For the `TENURE`, it's distribution is **right-skewed**. This may be due to the fact that a lot of clients are merely starting to become BPI clients and they happen to be young workers, which has great representation in the dataset.

### Credit Card Transactions

In [ ]:
credit_trans_cat_bar = ['MERCH_GROUP_cred', 'COUNTRY_cred']
plt.figure(figsize=(30, 30))
for i, c in enumerate(credit_trans_cat_bar):
  plt.subplot(3, 2, i+1)
  credit_trans[c].value_counts().sort_values(ascending=True).plot(
      kind='barh',
      title=f"Bar Graph of Frequency of {c}",
      ylabel=c,
      xlabel='Frequency',
      fontsize=8
  )

Retail Outlet Services, Miscellaneous Stores, and Transportation Services are the top three in `MERCH_GROUP`. This may be due to the fact that these stores and services are what people in general tend to do a lot of transactions on on a more regular basis.

In [ ]:
sorted = credit_trans['CHANNEL_cred'].value_counts().sort_values(ascending=False)
sorted.plot(kind='pie', fontsize = 9, autopct='%1.f%%', startangle=-270, labels=None, counterclock=False)
plt.title("Proportion of Channels of Credit Card Transactions", weight="bold")
plt.legend(
    labels=sorted.rename(lambda x: x.title().replace('_', ' ').replace('Pos', 'POS').replace('Sip', 'SIP')).index,
    bbox_to_anchor=(1, 1)
)
plt.ylabel('')
plt.tight_layout()
plt.show()

Since majority fall under traditional in `DIGITAL_FLAG`, it is no surprise that POS-Retail Purchase has a huge proportion.

In [ ]:
credit_trans['AMOUNT_cred'].plot(kind='hist', title="Histogram of AMOUNT_cred", xlabel = 'AMOUNT_cred')
plt.show()

Let's remove outliers to see distribution at lower amounts better.

In [ ]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

cred_no_outliers = remove_outliers(credit_trans, 'AMOUNT_cred')
cred_no_outliers['AMOUNT_cred'].plot(kind='hist', title="Histogram of AMOUNT_cred")
plt.show()


A lot of people perform transactions many times at small amounts but exponentially drop at higher amounts. This is the same case for debit, financial digital, incoming instapay, and outgoing instapay transactions.

### Debit Card Transactions

In [ ]:
sorted = debit_trans['CHANNEL_deb'].value_counts().sort_values(ascending=False)
sorted.plot(kind='pie', fontsize = 9, autopct='%1.f%%', startangle=-270, labels=None, counterclock=False)
plt.title("Proportion of Channels of Debit Card Transactions", weight="bold")
plt.legend(
    labels=sorted.rename(lambda x: x.title().replace('_', ' ').replace('Pos', 'POS')).index,
    bbox_to_anchor=(1, 1)
)
plt.ylabel('')
plt.tight_layout()
plt.show()

In [ ]:
debit_trans['AMOUNT_deb'].plot(kind='hist', title="Histogram of AMOUNT_deb", xlabel='AMOUNT_deb')
plt.show()

Let's remove outliers to see distribution at lower amounts better.

In [ ]:
deb_no_outliers = remove_outliers(debit_trans, 'AMOUNT_deb')
deb_no_outliers['AMOUNT_deb'].plot(kind='hist', title="Histogram of AMOUNT_deb")
plt.show()

### Financial Digital Transactions

In [ ]:
fin_trans['TRAN_TYPE_fin'].value_counts().sort_values(ascending=True).plot(
    kind='barh',
    title="Bar Graph of Frequency of TRAN_TYPE_fin",
    ylabel='TRAN_TYPE_fin',
    xlabel='Frequency',
    fontsize=8
)
plt.show()

In [ ]:
sorted = fin_trans['CHANNEL_fin'].value_counts().sort_values(ascending=False)
sorted.plot(kind='pie', fontsize = 9, autopct='%1.f%%', startangle=-270, labels=None, counterclock=False)
plt.title("Proportion of Channels of Financial Digital Transactions", weight="bold")
plt.legend(
    labels=sorted.rename(lambda x: x.title().replace('_', ' ').replace('Pos', 'Physical')).index,
    bbox_to_anchor=(1, 1)
)
plt.ylabel('')
plt.tight_layout()
plt.show()

Financial Digital Transactions through Web dominate the dataset. It is surprising that it is not through Mobile considering that there are a lot of cellphone users in the Philippines.

In [ ]:
fin_trans['AMOUNT_fin'].plot(kind='hist', title="Histogram of AMOUNT_fin", xlabel = 'AMOUNT_fin')
plt.show()

Let's remove outliers to see distribution at lower amounts better.

In [ ]:
fin_no_outliers = remove_outliers(fin_trans, 'AMOUNT_fin')
fin_no_outliers['AMOUNT_fin'].plot(kind='hist', title="Histogram of AMOUNT_fin")
plt.show()

### Incoming Instapay Transactions

In [ ]:
sorted = inc_inst['BPI_ACCT_TYPE_inc_inst'].value_counts().sort_values(ascending=False)
sorted.plot(kind='pie', fontsize = 9, autopct='%1.f%%', startangle=-270, labels=None, counterclock=False)
plt.title("Proportion of BPI Account Type of Incoming INSTAPAY Transactions", weight="bold")
plt.legend(
    labels=sorted.rename(lambda x: x.title().replace('Sa', 'Savings Account').replace('Ca', 'Checking Account')).index,
    bbox_to_anchor=(1, 1)
)
plt.ylabel('')
plt.tight_layout()
plt.show()

Almost every client uses a savings account. This maybe due to the fact that it is very expensive to open and maintain a checking account [(source)](https://www.moneymax.ph/personal-finance/articles/savings-account-vs-current-account).
 It could also be in the culture of most Filipinos to save and earn interest via a savings account.

In [ ]:
inc_inst['SRC_BANK_NAME_inc_inst'].value_counts().sort_values(ascending=True).plot(
    kind='barh',
    title="Bar Graph of Frequency of SRC_BANK_NAME_inc_inst",
    ylabel='SRC_BANK_NAME_inc_inst',
    xlabel='Frequency',
    fontsize=8
)
plt.show()

In [ ]:
inc_inst['AMOUNT_inc_inst'].plot(kind='hist', title="Histogram of AMOUNT_inc_inst", xlabel = 'AMOUNT_inc_inst')
plt.show()

### Outgoing Instapay Transactions

In [ ]:
out_inst['RCV_BANK_NAME_out_inst'].value_counts().sort_values(ascending=True).plot(
    kind='barh',
    title="Bar Graph of Frequency of RCV_BANK_NAME_out_inst",
    ylabel='SRC_BANK_NAME_out_inst',
    xlabel='Frequency',
    fontsize=8
)
plt.show()

The Human Bank is greatly represented in the banks of the source of incoming transactions. This suggests that BPI has one key competitor to watch out for.

In [ ]:
sorted = out_inst['CHANNEL_out_inst'].value_counts().sort_values(ascending=False)
sorted.plot(kind='pie', fontsize = 9, autopct='%1.f%%', startangle=-270, labels=None, counterclock=False)
plt.title("Proportion of BPI Account Type of Outgoing INSTAPAY Transactions", weight="bold")
plt.legend(labels=sorted.rename(lambda x: x.title()).index, bbox_to_anchor=(1, 1))
plt.ylabel('')
plt.tight_layout()
plt.show()

Mobile app must be more popular when it comes to Instapay transactions.

In [ ]:
out_inst['AMOUNT_out_inst'].plot(kind='hist', title="Histogram of AMOUNT_out_inst", xlabel = 'AMOUNT_out_inst')
plt.show()

## Bivariate Analysis

### Individual Datasets

#### Customer Information

In [ ]:
sns.scatterplot(x='AGE', y='TENURE', data=cust_info)
plt.title('TENURE vs. AGE')
plt.show()

In [ ]:
correlation = cust_info['AGE'].corr(cust_info['TENURE'])

print(f"Correlation between AGE and TENURE: {correlation}")

Low positive correlation between Age and Tenure.

In [ ]:
def boxes(x,y, df, rotation=0, figsize = (10,6)):
    fig, ax = plt.subplots(figsize=figsize)
    sns.boxplot(x=x,y=y, data=df)
    plt.xticks(rotation=rotation)
    plt.title('{ytitle} vs {xtitle}'.format(ytitle = y, xtitle = x))
    fig.subplots_adjust(bottom=0.2)
    plt.tight_layout()

boxes('AGE','INCOME_SOURCE',  cust_info)

In [ ]:
boxes('AGE','DIGITAL_FLAG',  cust_info)

There are less digital savvy people at older age.

In [ ]:
boxes('TENURE','MARITAL_STATUS',  cust_info)

Almost same distribution of Marital Status. There are many outliers at Single and Married.

In [ ]:
sns.countplot(data=cust_info, x='EDUCATION', hue='DIGITAL_FLAG')
plt.title('EDUCATION by DIGITAL_FLAG')
plt.show()

Regardless of Education, Traditional is still more preferred. But, there is high number of clients who prefer Digital.

#### Credit Card Transactions

In [ ]:
sns.countplot(data=credit_trans, x='MERCH_GROUP_cred', hue='CHANNEL_cred')
plt.xticks(rotation=90)
plt.title('MERCH_GROUP_cred by CHANNEL_cred')
plt.show()

POS-Retail Purchase is most frequent in Retail Outlet Services.

In [ ]:
boxes('AMOUNT_cred','CHANNEL_cred',  credit_trans)

Let's remove outliers to see distributions at lower amounts better.

In [ ]:
cred_no_out = remove_outliers(credit_trans, 'AMOUNT_cred')
boxes('AMOUNT_cred','CHANNEL_cred',  cred_no_out)

The highest credit card transaction amounts are mostly spent in SIP-Retail Purchase.

In [ ]:
boxes('AMOUNT_cred','MERCH_GROUP_cred', credit_trans)

Let's remove outliers to see distributions at lower amounts better.

In [ ]:
cred_no_out = remove_outliers(credit_trans, 'AMOUNT_cred')
boxes('AMOUNT_cred','MERCH_GROUP_cred', cred_no_out)

The distributions of Retail Outlet Services, Miscellaneous Stores, and Transportation Services fall around less than P5,000. But they also have lots of outiers.

#### Debit Card Transactions

In [ ]:
boxes('AMOUNT_deb', 'CHANNEL_deb', debit_trans)

Let's remove outliers to see distributions at lower amounts better.

In [ ]:
deb_no_out = remove_outliers(debit_trans, 'AMOUNT_deb')
boxes('AMOUNT_deb', 'CHANNEL_deb',  deb_no_out)

The highest debit card transaction amounts are thru POS.

#### Financial Digital Transactions

In [ ]:
sns.countplot(data = fin_trans, x='TRAN_TYPE_fin', hue='CHANNEL_fin')
plt.xticks(rotation=90)
plt.title('MERCH_GROUP_fin by CHANNEL_fin')
plt.show()

WEB is mostly used with E-Wallet. MOBILE is mostly used in Bills Payment and Internal Funds Transfer.

In [ ]:
boxes('AMOUNT_fin', 'CHANNEL_fin', fin_trans)

Let's remove outliers to see distributions at lower amounts better.

In [ ]:
fin_no_out = remove_outliers(fin_trans, 'AMOUNT_fin')
boxes('AMOUNT_fin', 'CHANNEL_fin', fin_no_out)

Although beams only constitute around 1% of the proportion, its distribution is the widest.

In [ ]:
boxes('AMOUNT_fin', 'TRAN_TYPE_fin', fin_trans)

Let's remove outliers to see distributions at lower amounts better.



In [ ]:
fin_no_out = remove_outliers(fin_trans, 'AMOUNT_fin')
boxes('AMOUNT_fin', 'TRAN_TYPE_fin', fin_no_out)

The highest financial digital transaction amounts are thru INTERNAL FUNDS TRANSFER or between one BPI client to another BPI client.

#### Incoming Instapay Transactions

In [ ]:
sns.countplot(data=inc_inst, x='SRC_BANK_NAME_inc_inst', hue='BPI_ACCT_TYPE_inc_inst')
plt.xticks(rotation=90)
plt.title('SRC_BANK_NAME_inc_inst by BPI_ACCT_TYPE_inc_inst')
plt.show()

Checking Account is almost not used by source.

In [ ]:
boxes('AMOUNT_inc_inst', 'SRC_BANK_NAME_inc_inst',  inc_inst, figsize=(10, 10))

In [ ]:
inc_inst_no_out = remove_outliers(inc_inst, 'AMOUNT_inc_inst')
boxes('AMOUNT_inc_inst', 'BPI_ACCT_TYPE_inc_inst', inc_inst_no_out)

#### Outgoing Instapay Transactions

In [ ]:
sns.countplot(data=out_inst, x='RCV_BANK_NAME_out_inst', hue='CHANNEL_out_inst')
plt.xticks(rotation=90)
plt.title('RCV_BANK_NAME_out_inst by CHANNEL_out_inst')
plt.show()

`ONLINE` is almost not used by recipient.

In [ ]:
boxes('AMOUNT_out_inst', "CHANNEL_out_inst", out_inst)

MOBILE is used more in higher transactions, but its distribution is concentrated at slightly lower transaction amounts than ONLINE.

In [ ]:
boxes('AMOUNT_out_inst', 'RCV_BANK_NAME_out_inst', out_inst, figsize=(10, 10))

## Multivariate Analysis

#### RFM (Feature Engineering)

In the datasets other than `CUSTOMER INFO_PJDSC`, the lengths of `CUST_NUM` are less than the unique values of `CUST_NUM`. The dates and transaction amounts differ in each unique `CUST_NUM`. So we created a new dataframe from each dataset with Recency, Frequency, and Total Transaction Amount. In this way, the `CUST_NUM` would be unique for the datasets other than `CUSTOMER INFO_PJDSC`. [(source)](https://connectif.ai/en/blog/what-are-rfm-scores-and-how-to-calculate-them/)

Lower Recency means most recent. Convert 0 Recency to 1 to mitigate -inf problem in logarithmic transformation used in modelling.

In [ ]:
def rep_0(i):
  return 1 if i == 0 else i

##### Credit Card Transactions

In [ ]:
credit_trans['DATE_max_cred'] = credit_trans['DATE_cred']
credit_trans['DATE_min_cred'] = credit_trans['DATE_cred']
credit_trans['Frequency_cred'] = credit_trans['CUST_NUM']
reference_date = datetime(2023, 6, 30)

credit_trans_rfm = credit_trans.groupby("CUST_NUM").agg({
    "Frequency_cred": "count",
    "AMOUNT_cred": "sum",
    "DATE_max_cred": "max"
})

credit_trans_rfm = credit_trans_rfm.reset_index()

In [ ]:
credit_trans_rfm['Recency_cred'] = reference_date - credit_trans_rfm["DATE_max_cred"]
credit_trans_rfm['Recency_cred'] = credit_trans_rfm['Recency_cred'].astype(str)
credit_trans_rfm['Recency_cred'] = credit_trans_rfm['Recency_cred'].str.replace('days', '')
credit_trans_rfm['Recency_cred'].info()
credit_trans_rfm['Recency_cred'] = credit_trans_rfm['Recency_cred'].astype(int)
credit_trans_rfm.drop(columns=['DATE_max_cred'], inplace=True)
credit_trans_rfm['Recency_cred'] = credit_trans_rfm['Recency_cred'].apply(rep_0)

credit_trans_rfm.head()

##### Debit Card Transactions

In [ ]:
debit_trans['DATE_max_deb'] = debit_trans['DATE_deb']
debit_trans['DATE_min_deb'] = debit_trans['DATE_deb']
debit_trans['Frequency_deb'] = debit_trans['CUST_NUM']
reference_date = datetime(2023, 6, 30)

debit_trans_rfm = debit_trans.groupby("CUST_NUM").agg({
    "Frequency_deb" : "count",
    "AMOUNT_deb" : "sum",
    "DATE_max_deb" : "max"
})

debit_trans_rfm = debit_trans_rfm.reset_index()

In [ ]:
debit_trans_rfm['Recency_deb'] = reference_date - debit_trans_rfm["DATE_max_deb"]
debit_trans_rfm['Recency_deb'] = debit_trans_rfm['Recency_deb'].astype(str)
debit_trans_rfm['Recency_deb'] = debit_trans_rfm['Recency_deb'].str.replace('days', '')
debit_trans_rfm['Recency_deb'] = debit_trans_rfm['Recency_deb'].astype(int)
debit_trans_rfm.drop(columns=['DATE_max_deb'], inplace=True)
debit_trans_rfm['Recency_deb'] = debit_trans_rfm['Recency_deb'].apply(rep_0)

debit_trans_rfm.head()

##### Financial Digital Transactions

In [ ]:
fin_trans['DATE_max_fin'] = fin_trans['DATE_fin']
fin_trans['DATE_min_fin'] = fin_trans['DATE_fin']
fin_trans['Frequency_fin'] = fin_trans['CUST_NUM']
reference_date = datetime(2023, 6, 30)

fin_trans_rfm = fin_trans.groupby("CUST_NUM").agg({
    "Frequency_fin": "count",
    "AMOUNT_fin": "sum",
    "DATE_max_fin": "max"
})

fin_trans_rfm = fin_trans_rfm.reset_index()

In [ ]:
fin_trans_rfm['Recency_fin'] = reference_date - fin_trans_rfm["DATE_max_fin"]
fin_trans_rfm['Recency_fin'] = fin_trans_rfm['Recency_fin'].astype(str)
fin_trans_rfm['Recency_fin'] = fin_trans_rfm['Recency_fin'].str.replace('days', '')
fin_trans_rfm['Recency_fin'] = fin_trans_rfm['Recency_fin'].astype(int)
fin_trans_rfm.drop(columns=['DATE_max_fin'], inplace=True)
fin_trans_rfm['Recency_fin'] = fin_trans_rfm['Recency_fin'].apply(rep_0)

fin_trans_rfm.head()

##### Incoming Instapay Transactions

In [ ]:
inc_inst['DATE_max_inc_inst'] = inc_inst['DATE_inc_inst']
inc_inst['DATE_min_inc_inst'] = inc_inst['DATE_inc_inst']
inc_inst['Frequency_inc_inst'] = inc_inst['CUST_NUM']
reference_date = datetime(2023,6,30)

inc_inst_rfm = inc_inst.groupby("CUST_NUM").agg({
    "Frequency_inc_inst": "count",
    "AMOUNT_inc_inst": "sum",
    "DATE_max_inc_inst": "max"
})

inc_inst_rfm = inc_inst_rfm.reset_index()

In [ ]:
inc_inst_rfm['Recency_inc_inst'] = reference_date - inc_inst_rfm["DATE_max_inc_inst"]
inc_inst_rfm['Recency_inc_inst'] = inc_inst_rfm['Recency_inc_inst'].astype(str)
inc_inst_rfm['Recency_inc_inst'] = inc_inst_rfm['Recency_inc_inst'].str.replace('days','')
inc_inst_rfm['Recency_inc_inst'] = inc_inst_rfm['Recency_inc_inst'].astype(int)
inc_inst_rfm.drop(columns=['DATE_max_inc_inst'], inplace=True)
inc_inst_rfm['Recency_inc_inst'] = inc_inst_rfm['Recency_inc_inst'].apply(rep_0)

inc_inst_rfm.head()

##### Outgoing Instapay Transactions

In [ ]:
out_inst['DATE_max_out_inst'] = out_inst['DATE_out_inst']
out_inst['DATE_min_out_inst'] = out_inst['DATE_out_inst']
out_inst['Frequency_out_inst'] = out_inst['CUST_NUM']
reference_date = datetime(2023, 6, 30)


out_inst_rfm = out_inst.groupby("CUST_NUM").agg({
    "Frequency_out_inst": "count",
    "AMOUNT_out_inst": "sum",
    "DATE_max_out_inst": "max"
})

out_inst_rfm = out_inst_rfm.reset_index()

In [ ]:
out_inst_rfm['Recency_out_inst'] = reference_date - out_inst_rfm["DATE_max_out_inst"]
out_inst_rfm['Recency_out_inst'] = out_inst_rfm['Recency_out_inst'].astype(str)
out_inst_rfm['Recency_out_inst'] = out_inst_rfm['Recency_out_inst'].str.replace('days','')
out_inst_rfm['Recency_out_inst'] = out_inst_rfm['Recency_out_inst'].astype(int)
out_inst_rfm.drop(columns=['DATE_max_out_inst'], inplace=True)
out_inst_rfm['Recency_out_inst']=out_inst_rfm['Recency_out_inst'].apply(rep_0)

out_inst_rfm.head()

#### Merge Datasets

In [ ]:
merged_df = cust_info.set_index('CUST_NUM').merge(credit_trans_rfm.set_index('CUST_NUM'), left_index=True, right_index=True, how='outer')
merged_df = merged_df.merge(debit_trans_rfm.set_index('CUST_NUM'), left_index=True, right_index=True, how='outer')
merged_df = merged_df.merge(fin_trans_rfm.set_index('CUST_NUM'), left_index=True, right_index=True, how='outer')
merged_df = merged_df.merge(inc_inst_rfm.set_index('CUST_NUM'), left_index=True, right_index=True, how='outer')
merged_df = merged_df.merge(out_inst_rfm.set_index('CUST_NUM'), left_index=True, right_index=True, how='outer')
merged_df.head()

In [ ]:
merged_df.shape

Number of rows is exactly the same as the number of rows of cleaned customer info dataset.

In [ ]:
merged_df.info()

In [ ]:
merged_df_nulls = merged_df.replace('NO_DATA', np.nan, inplace=False).isnull().sum()
merged_df_nulls

In [ ]:
merged_df_nulls.plot(kind='bar')
plt.show()

In [ ]:
total_merged_df = len(merged_df)
(merged_df_nulls / total_merged_df) * 100

There is null in almost every row.

In [ ]:
merged_df.dropna().shape

In [ ]:
selected_features = [
    'AGE',
    'TENURE',
    'Frequency_cred',
    'AMOUNT_cred',
    'Recency_cred',
    'Frequency_deb',
    'AMOUNT_deb',
    'Recency_deb',
    'Frequency_fin',
    'AMOUNT_fin',
    'Recency_fin',
    'Frequency_inc_inst',
    'AMOUNT_inc_inst',
    'Recency_inc_inst',
    'Frequency_out_inst',
    'AMOUNT_out_inst',
    'Recency_out_inst'
  ]

In [ ]:
corr_matr = merged_df[selected_features].corr(method='pearson')
plt.figure(figsize=(10,10))
sns.heatmap(corr_matr, annot=True, cmap='coolwarm', square=True)
plt.title("Pearson's Correlation Heatmap of Numerical Features")
plt.show()

Moderately positive correlations:<br>
* `AGE` and `TENURE` - longer tenure => more likely to be older<br>
* `AMOUNT_inc_inst` and `AMOUNT_deb`- more likely to spend if higher incoming money<br>
* `AMOUNT_inc_inst` and `AMOUNT_out_inst` - more likely to spend if higher incoming money<br>
* `AMOUNT_deb` and `Frequency_deb` - more transactions, higher amount<br>
* `AMOUNT_inc_inst` and `Frequency_inc_inst` - more transactions, higher amount<br>
* `AMOUNT_inc_inst` and `Frequency_out_inst` - more transactions, higher amount<br>
* `AMOUNT_inc_inst` and `AMOUNT_fin`- higher incoming, more likely to have more digital transactions <br>

In [ ]:
rfms = [
    'Recency_cred',
    'Recency_deb',
    'Recency_fin',
    'Recency_inc_inst',
    'Recency_out_inst',
    'Frequency_cred',
    'Frequency_deb',
    'Frequency_fin',
    'Frequency_inc_inst',
    'Frequency_out_inst'
]

for r in rfms:
  outliers = perc_outliers(merged_df, r)
  print(f'Percentage of outliers for {r}: {outliers:.2f}%')

Can drop outliers in Recency and Frequency.

#### Merged Datasets Analysis

In [ ]:
def multivariate_plots(df, x, y):
    key_features = ['INCOME_SOURCE', 'EDUCATION', 'GENDER', 'MARITAL_STATUS', 'DIGITAL_FLAG']
    plt.figure(figsize=(8, 5))
    for i, c in enumerate(key_features):
        scatter = sns.scatterplot(
            x=x,
            y=y,
            hue=c,
            size=1,
            sizes=[10],
            data=df,
            palette='colorblind'
        )
        handles, labels = scatter.get_legend_handles_labels()

        plt.legend(handles=handles[:-1], title=c, loc='upper left', bbox_to_anchor=(1, 1), scatterpoints=1)
        plt.title(f'{y} by {c} vs {x}')
        plt.show()

We explore how `INCOME_SOURCE`, `EDUCATION`, `GENDER`, `MARITAL_STATUS`, and `DIGITAL_FLAG` relate to the amounts, frequencies, and recencies with varying ages and lengths of tenure.

In [ ]:
def nest_multivariate(df, x):
      main_features = ['AMOUNT_cred', 'AMOUNT_deb', 'AMOUNT_fin', 'AMOUNT_inc_inst', 'AMOUNT_out_inst']
      for i, c in enumerate(main_features):
        no_outliers = remove_outliers(df, c)
        multivariate_plots(no_outliers, x, c)

We'll remove outliers in the amounts column for better visualization.


##### Median and Mean Bar Graphs (Bivariate)

Since most of the amounts have huge proportion of outliers, we will use median on them. For recency and frequency, we will use mean.

In [ ]:
def plot_median(x, y):
  col_dict = {}
  cols = merged_df[x].unique().tolist()

  for s in cols:
    col_dict[s] = merged_df[merged_df[x] == s][y].median()

  df = pd.DataFrame.from_dict(col_dict, orient='index', columns=['median'])
  df.reset_index(inplace=True)
  plt.bar(df['index'], df['median'])
  plt.ylabel(f'median {y}')
  plt.xlabel(x)
  plt.title(f'{y} vs {x}')
  plt.xticks(rotation=90)
  plt.show()

In [ ]:
def plot_mean(x, y):
  col_dict = {}
  cols = merged_df[x].unique().tolist()

  for s in cols:
    col_dict[s] = merged_df[merged_df[x] == s][y].mean()

  df = pd.DataFrame.from_dict(col_dict, orient='index', columns=['mean'])
  df.reset_index(inplace=True)
  plt.bar(df['index'], df['mean'])
  plt.ylabel(f'mean {y}')
  plt.xlabel(x)
  plt.title(f'{y} vs {x}')
  plt.xticks(rotation=90)
  plt.show()

In [ ]:
y_rec_freq= ['Recency_deb', 'Recency_cred', 'Recency_fin', 'Recency_inc_inst', 'Recency_out_inst',
              'Frequency_deb', 'Frequency_cred', 'Frequency_fin', 'Frequency_inc_inst', 'Frequency_out_inst']

In [ ]:
y_amounts = ['AMOUNT_deb', 'AMOUNT_cred', 'AMOUNT_fin', 'AMOUNT_inc_inst', 'AMOUNT_out_inst']

In [ ]:
x_interest = ['INCOME_SOURCE', 'EDUCATION', 'GENDER', 'MARITAL_STATUS', 'DIGITAL_FLAG']

###### Median

In [ ]:
for x in x_interest:
  for y in y_amounts:
    plot_median(x, y)

###### Mean

In [ ]:
for x in x_interest:
  for y in y_rec_freq:
    plot_mean(x, y)

Based on this, `INCOME_SOURCE` and `EDUCATION` appear to be great candidates for features that can be used for clustering due to their varying nature. `GENDER`, `MARITAL_STATUS`, and `DIGITAL_FLAG` appear to be equally distributed or obvious.

##### Age vs. Amounts

In [ ]:
nest_multivariate(merged_df, 'AGE')

##### Age vs. Recencies and Frequencies

In [ ]:
#Recencies
rec_list = [
    'Recency_cred',
    'Recency_deb',
    'Recency_fin',
    'Recency_inc_inst',
    'Recency_out_inst'
]

for elt in rec_list:
  multivariate_plots(merged_df, 'AGE', elt)

In [ ]:
#Frequencies
freq_list = [
    'Frequency_cred',
    'Frequency_deb',
    'Frequency_fin',
    'Frequency_inc_inst',
    'Frequency_out_inst'
  ]

for elt in freq_list:
  freq_no_outliers = remove_outliers(merged_df, elt)
  multivariate_plots(freq_no_outliers, 'AGE', elt)

#####Tenure vs. Amounts

In [ ]:
nest_multivariate(merged_df, 'TENURE')

##### Tenure vs. Recencies and Frequencies

In [ ]:
#Recencies
rec_list = [
    'Recency_cred',
    'Recency_deb',
    'Recency_fin',
    'Recency_inc_inst',
    'Recency_out_inst'
]

for elt in rec_list:
  multivariate_plots(merged_df, 'TENURE', elt)

In [ ]:
#Frequencies
freq_list = [
    'Frequency_cred',
    'Frequency_deb',
    'Frequency_fin',
    'Frequency_inc_inst',
    'Frequency_out_inst'
]

for elt in freq_list:
  freq_no_outliers = remove_outliers(merged_df, elt)
  multivariate_plots(freq_no_outliers, 'TENURE', elt)

##### Amounts vs. Each other

###### AMOUNT_deb vs. AMOUNT_cred

In [ ]:
cred_no_outliers = remove_outliers(merged_df, 'AMOUNT_cred')
cred_deb_no_outliers = remove_outliers(cred_no_outliers, 'AMOUNT_deb')
multivariate_plots(cred_deb_no_outliers, 'AMOUNT_cred', 'AMOUNT_deb')

###### AMOUNT_inc_inst vs. AMOUNT_out_inst

In [ ]:
inc_no_outliers = remove_outliers(merged_df, 'AMOUNT_inc_inst')
inc_out_no_outliers = remove_outliers(inc_no_outliers, 'AMOUNT_out_inst')
multivariate_plots(inc_out_no_outliers, 'AMOUNT_inc_inst', 'AMOUNT_out_inst')

###### AMOUNT_deb vs. AMOUNT_out_inst

In [ ]:
deb_no_outliers = remove_outliers(merged_df, 'AMOUNT_deb')
deb_out_no_outliers = remove_outliers(deb_no_outliers, 'AMOUNT_out_inst')
multivariate_plots(deb_out_no_outliers, 'AMOUNT_deb', 'AMOUNT_out_inst')

###### AMOUNT_inc_inst vs. AMOUNT_out_inst vs AMOUNT_fin

In [ ]:
inc_no_outliers = remove_outliers(merged_df, 'AMOUNT_inc_inst')
inc_out_no_outliers = remove_outliers(inc_no_outliers, 'AMOUNT_out_inst')
inc_out_fin_no_outliers = remove_outliers(inc_out_no_outliers, 'AMOUNT_fin')

fig = px.scatter_3d(
    data_frame=inc_out_fin_no_outliers,
    x="AMOUNT_inc_inst",
    y="AMOUNT_out_inst",
    z="AMOUNT_fin",
    height=550,
    title="AMOUNT_inc_inst vs. AMOUNT_out_inst vs AMOUNT_fin"
)
fig.update_traces(marker={'size': 1})
fig.show()

###### AMOUNT_inc_inst vs. AMOUNT_deb_inst vs AMOUNT_fin

In [ ]:
inc_no_outliers = remove_outliers(merged_df, 'AMOUNT_inc_inst')
inc_deb_no_outliers = remove_outliers(inc_no_outliers, 'AMOUNT_deb')
inc_deb_fin_no_outliers = remove_outliers(inc_deb_no_outliers, 'AMOUNT_fin')

fig = px.scatter_3d(
    data_frame=inc_deb_fin_no_outliers,
    x="AMOUNT_inc_inst",
    y="AMOUNT_deb",
    z="AMOUNT_fin",
    height=550,
    title = "AMOUNT_inc_inst vs. AMOUNT_deb vs AMOUNT_fin"
)
fig.update_traces(marker={'size': 1})
fig.show()

##### Insights

`INCOME_SOURCE` and `EDUCATION` are good candidates for features to be used in clustering as there are apparent distinctions that can be seen in the plots. `GENDER` and `MARITAL_STATUS` are equally represented when compared with the amounts, recencies, and frequencies. `DIGITAL_FLAG` is somewhat correlated with `AGE` since younger people tend to be more digital savvy. Furthermore, people tend to be transact more using INSTAPAY and digitally when they are digitally savvy. Hence, `GENDER`, `MARITAL_STATUS`, and `DIGITAL_FLAG` won't be used in clustering.

# Clustering Algorithm

## Data Processing

Features of Interest: Age, Income Source, Education, Spending Habits (Debit Transaction Amounts, Recencies, Frequencies)<br><br>
We chose to not include Tenure since we do not want the results to simply cater to BPI clients, but to people in far-flung areas as a whole. Age is also somewhat correlated to Tenure, which would cause multicollinearity in clustering. <br><br>
Segmenting according to these features is done because they appear to be key indicators that differentiate one group of clients to another. We specifically chose the Debit Card transactions over the others because aside from the fact that they make up a huge portion of the whole customer database with over ~20k rows in the merged dataframe, creating products that favor the masses is the primary goal of our EDA.<br><br>
KMeans Clustering Algorithm is performed. This is performed because not only is it popular among data scientists, it has also been proven to be effective by many studies. It is simple to implement, it can handle large datasets like the Debit Card Transactions dataset, and the clusters it forms are easy to interpret [(source)](https://developers.google.com/machine-learning/clustering/algorithm/advantages-disadvantages).

Drop outliers in Recency and Frequency, and Age.
Perform RandomForestClassifier on EDUCATION.
Drop missing values in AGE and INCOME_SOURCE.

### Feature Selection

In [ ]:
deb_model_features = ["AGE", "INCOME_SOURCE", "EDUCATION", "AMOUNT_deb", "Recency_deb", "Frequency_deb"]

In [ ]:
model_df = merged_df[deb_model_features]
model_df = model_df.replace('NO_DATA', np.nan, inplace=False)
model_df.head()

In [ ]:
model_df.info()

### Convert Categorical to Numerical

We convert Categorical to Numerical since KMeans only interpret numerical values. [(source)](https://medium.com/@evgen.ryzhkov/5-stages-of-data-preprocessing-for-k-means-clustering-b755426f9932)

#### INCOME_SOURCE using One-Hot Encoding

One-Hot Encoding is appropriate for `INCOME_SOURCE` since it is not ordinal data. [(source)](https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/)

We first convert the 'NO_DATA' into NaN so that handling null values would be easier to handle later.

In [ ]:
model_df = model_df.replace('NO_DATA', np.nan)

In [ ]:
model_df.isnull().sum()

In [ ]:
model_df = pd.get_dummies(model_df, columns=['INCOME_SOURCE'], prefix = 'INCOME', dummy_na=True)

In [ ]:
model_df.head()

In [ ]:
total_model_df = len(model_df)
(len(model_df[model_df['INCOME_nan'] == 1]) / total_model_df) * 100

#### EDUCATION using Label Encoding

Label Encoding is appropriate for `EDUCATION` since it is ordinal data. [(source)](https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/).

In [ ]:
custom_labels = {np.nan: 0, 'LOW': 1, 'MID': 2, 'HIGH': 3}
model_df['EDUCATION'] = model_df['EDUCATION'].map(custom_labels)
model_df.head()

In [ ]:
model_df['EDUCATION'].value_counts().plot(kind='pie')

### Null Values Treatment

We only handle the null values of the features of interest. These are Age, Income Source, Education, and Spending Habits (Debit Transaction Amounts, Recencies, Frequencies). There should be no null values since KMeans cannot handle null values. [(source)](https://www.linkedin.com/advice/3/how-do-you-validate-compare-clustering-results-missing)

In [ ]:
model_df['EDUCATION'] = model_df['EDUCATION'].replace(0, np.nan)
model_df.head()

#### Drop Nulls in AMOUNT_deb, Recency_deb, and Frequency_deb.

By dropping nulls in `AMOUNT_deb`, `Recency_deb`, and `Frequency_deb`, since the Debit Card Transactions dataset does not have any nulls, the nulls that stem from the other datasets would be removed.

In [ ]:
total_model_df = len(model_df)
(model_df.isnull().sum() / total_model_df) * 100

In [ ]:
columns_to_check = ["AMOUNT_deb", "Recency_deb", "Frequency_deb"]
model_df.dropna(subset=columns_to_check, inplace=True)

In [ ]:
model_df.shape

In [ ]:
total_model_df = len(model_df)
(model_df.isnull().sum() / total_model_df) * 100

#### Drop Nulls in AGE and INCOME_SOURCE

Since there are a few nulls in `INCOME_SOURCE` and `AGE`, it is best to simply drop them.

In [ ]:
columns_to_check = ["AGE"]
model_df.dropna(subset=columns_to_check, inplace=True)

In [ ]:
total_model_df = model_df.shape[0]
(model_df.isnull().sum() / total_model_df) * 100

In [ ]:
total_model_df

In [ ]:
#Remove nulls in INCOME_SOURCE
model_df = model_df[model_df['INCOME_nan'] != 1]
model_df.shape[0]

In [ ]:
model_df.drop(columns=['INCOME_nan'], inplace=True)
model_df.shape

In [ ]:
model_df.head()

In [ ]:
total_model_df = model_df.shape[0]
(model_df.isnull().sum() / total_model_df) * 100

#### EDUCATION Imputation using Random Forest Classifier.

Since there are a lot of nulls in `EDUCATION`, we have decided to use Random Forest Classifier to predict the education levels of the clients that do not have the education data and to take care of the huge number of nulls.

In [ ]:
model_df['EDUCATION'].value_counts().plot(kind='bar')

In [ ]:
model_df['EDUCATION'].value_counts().plot(kind='pie')

In [ ]:
model_df_filled = model_df.dropna(subset=['EDUCATION'])

X = model_df_filled.drop('EDUCATION', axis=1)
y = model_df_filled['EDUCATION']

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X, y)

missing_education = model_df.loc[model_df['EDUCATION'].isnull(), X.columns]
imputed_education = rf_classifier.predict(missing_education)

model_df.loc[model_df['EDUCATION'].isnull(), 'EDUCATION'] = imputed_education

In [ ]:
model_df['EDUCATION'].value_counts().plot(kind='bar')

In [ ]:
model_df['EDUCATION'].value_counts().plot(kind='pie')

In [ ]:
model_df.isnull().sum()

In [ ]:
model_df.head()

### Logarithmic Transformation

In [ ]:
def distributions(df):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1, figsize=(8,8))
    sns.histplot(df["AGE"], ax=ax1)
    sns.histplot(df["AMOUNT_deb"], ax=ax2)
    sns.histplot(df["Recency_deb"], ax=ax3)
    sns.histplot(df["Frequency_deb"], ax=ax4)
    plt.tight_layout()

In [ ]:
distributions(model_df)

Remove right-skewness using logarithmic transformation. This is because KMeans prefer to have normal distributions. This also reduces the impact of outliers.[(source)](https://medium.com/@evgen.ryzhkov/5-stages-of-data-preprocessing-for-k-means-clustering-b755426f9932)

In [ ]:
model_df_log = np.log(model_df[["AGE", "AMOUNT_deb","Recency_deb","Frequency_deb"]])
distributions(model_df_log)

In [ ]:
model_df.head()

In [ ]:
model_df_copy = model_df.copy()
model_df_copy[['AGE','AMOUNT_deb','Recency_deb',	'Frequency_deb']] = model_df_log

In [ ]:
model_df_copy.head()

### Standardize

Standardize in order for the algorithm to consider each features equally. [(source)](https://medium.com/@evgen.ryzhkov/5-stages-of-data-preprocessing-for-k-means-clustering-b755426f9932)

In [ ]:
scaler = StandardScaler()
model_df_scaled = scaler.fit_transform(model_df_copy)
model_df_scaled

In [ ]:
model_df_scaled = pd.DataFrame(model_df_scaled, columns=model_df_copy.columns)
model_df_scaled.head()

## KMeans Algorithm (without PCA)

### Elbow and Silhouette Method

In [ ]:
kmeans_set = {"init": "random", "max_iter": 300, "random_state": 69, 'n_init': 10}

num_clusters = range(2,11)
inertias = []
silhouette_coefficients = []

for k in num_clusters:
    kmeans = KMeans(n_clusters=k, **kmeans_set)
    kmeans.fit(model_df_scaled)
    inertias.append(kmeans.inertia_)
    score=silhouette_score(model_df_scaled, kmeans.labels_)
    silhouette_coefficients.append(score)

In [ ]:
plt.style.use("seaborn-v0_8")
fig,ax=plt.subplots(1, 2, figsize=(20, 5))
kl = KneeLocator(range(2, 11), inertias, curve='convex', direction="decreasing")
ax[0].plot(range(2, 11), inertias,color='blue',marker='o',label='Elbow Method')
ax[0].set_xticks(range(2, 11))
ax[0].set_xlabel("Number of Clusters", labelpad=20)
ax[0].set_ylabel("Inertia", labelpad=20)
ax[0].set_title("Elbow Method")

ax[1].plot(range(2, 11), silhouette_coefficients, color='blue', marker='o', label='silhouette coefficients')
ax[1].set_xticks(range(2, 11))
ax[1].set_title("Silhouette Values")
ax[1].set_xlabel("Number of Clusters")
ax[1].set_ylabel("silhouette_coefficients")
plt.tight_layout()

We chose 7 clusters since the first elbow appears to be on 7, and it still has relatively high silhouette coefficient.

### Cluster Exploration

In [ ]:
def clusterboxes(x, y, h, data, r=45):
  fig, ax = plt.subplots(figsize=(10,6))
  box = sns.boxplot(x=x, y=y, hue=h, data=data)
  box.set_xticklabels(box.get_xticklabels(), rotation=r)
  fig.subplots_adjust(bottom=0.2)
  plt.tight_layout()

In [ ]:
kmeans = KMeans(n_clusters=7, **kmeans_set)
kmeans.fit(model_df_scaled)
model_df_scaled['Cluster'] = kmeans.labels_
model_df_scaled.head()

In [ ]:
model_df['Cluster'] = model_df_scaled['Cluster'].values
model_df.head()

In [ ]:
def multivariate_clustered_num(df, x, y):
    plt.figure(figsize=(8, 5))
    scatter = sns.scatterplot(
        x=x,
        y=y,
        hue='Cluster',
        size=1,
        sizes=[10],
        data=df,
        palette='Spectral'
    )
    handles, labels = scatter.get_legend_handles_labels()

    plt.legend(handles=handles[:-1], title='Cluster', loc='upper left', bbox_to_anchor=(1, 1), scatterpoints=1)
    plt.title('{y_value} by {c_value} vs {x_value}'.format(y_value=y, x_value=x, c_value='Cluster'))
    plt.show()

In [ ]:
sns.countplot(x = model_df['Cluster'])

In [ ]:
for num in range(6):
  cluster_length = len(model_df[model_df['Cluster'] == num])
  print('Length of Cluster {l}:'.format(l = num), cluster_length)

In [ ]:
no_outliers = remove_outliers(model_df, 'AMOUNT_deb')
multivariate_clustered_num(no_outliers, 'AGE', 'AMOUNT_deb')

In [ ]:
multivariate_clustered_num(model_df, 'AGE', 'Recency_deb')

In [ ]:
no_outliers = remove_outliers(model_df, 'Frequency_deb')
multivariate_clustered_num(no_outliers, 'AGE', 'Frequency_deb')

In [ ]:
incomes = model_df.columns[5:16]
for i in incomes:
  no_outliers = remove_outliers(model_df, 'AMOUNT_deb')
  clusterboxes(i, "AMOUNT_deb", "Cluster", no_outliers)

In [ ]:
no_outliers = remove_outliers(model_df, 'AMOUNT_deb')
clusterboxes("EDUCATION", "AMOUNT_deb", "Cluster", no_outliers)

In [ ]:
incomes = model_df.columns[5:16]
for i in incomes:
  sns.countplot(data=model_df, x=i, hue='Cluster')
  plt.title('{income} by Cluster'.format(income=i))
  plt.show()

In [ ]:
sns.countplot(data=model_df, x='EDUCATION', hue='Cluster')
plt.title('EDUCATION by Cluster')
plt.show()

### Cluster Characteristics

The following are the characteristics of each cluster (number at the end is the rank in terms of quantity):
*   **Cluster 0**: Middle-aged, mid-high spender, salaried (almost all), mostly mid educ, mid-high frequency, 1
*   **Cluster 1**: Scattered, mostly mid educ, 7
*   **Cluster 2**: Old, high spender, remittance people, mostly mid educ, 5
*   **Cluster 3**: Old, low spender, pension, mostly low educ, 4
*   **Cluster 4**: Young, mostly low educ, 3
*   **Cluster 5**: Middle-aged, low spender, salaried, mostly mid educ, low frequency, 2
*   **Cluster 6**: Scattered, business people, mostly mid educ, 6





## KMeans Algorithm (with PCA)

We attempted to use KMeans algorithm with Principal Component Analysis in order to reduce dimensionality. This was employed to see if there was any difference in clustering.

In [ ]:
pca_model_kmeans = model_df_scaled[model_df_scaled.columns.tolist()[:-1]]
pca_model_kmeans.head()

In [ ]:
pca = PCA()
pca.fit(pca_model_kmeans)
explained_variance_ratio = pca.explained_variance_ratio_

plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio.cumsum(), marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Explained Variance Ratio vs. Number of Components')
plt.show()

The "elbow point" in the plot is where the explained variance starts to level off. This is a good indicator of the number of components to retain. We chose 90%, hence 11 as the number of components.

In [ ]:
pca = PCA(n_components = 11)
X_principal = pca.fit_transform(pca_model_kmeans)
X_principal = pd.DataFrame(X_principal)
X_principal.columns = ['P1', 'P2','P3', 'P4','P5', 'P6','P7', 'P8','P9','P10', 'P11']

X_principal.head()

In [ ]:
X_principal.shape

In [ ]:
kmeans_set = {"init": "random", "max_iter": 300, "random_state": 69, 'n_init': 10}

num_clusters = range(2,11)
inertias = []
silhouette_coefficients = []

for k in num_clusters:
    kmeans = KMeans(n_clusters=k, **kmeans_set)
    kmeans.fit(X_principal)
    inertias.append(kmeans.inertia_)
    score=silhouette_score(X_principal, kmeans.labels_)
    silhouette_coefficients.append(score)

In [ ]:
fig,ax=plt.subplots(1, 2, figsize=(20, 5))
kl = KneeLocator(range(2, 11), inertias, curve='convex', direction="decreasing")
ax[0].plot(range(2, 11), inertias,color='blue',marker='o',label='Elbow Method')
ax[0].set_xticks(range(2, 11))
ax[0].set_xlabel("Number of Clusters", labelpad=20)
ax[0].set_ylabel("Inertia", labelpad=20)
ax[0].set_title("Elbow Method")

ax[1].plot(range(2, 11), silhouette_coefficients, color='blue', marker='o', label='silhouette coefficients')
ax[1].set_xticks(range(2, 11))
ax[1].set_title("Silhouette Values")
ax[1].set_xlabel("Number of Clusters")
ax[1].set_ylabel("silhouette_coefficients")
plt.tight_layout()

We chose 7 clusters because of the elbow method, and it still has relatively high silhouette coefficient; even higher than in the results for KMeans without PCA (from ~0.36 - ~0.43).

### Cluster Exploration

In [ ]:
kmeans = KMeans(n_clusters=7, **kmeans_set)
kmeans.fit(X_principal)
X_principal['Cluster'] = kmeans.labels_
X_principal.head()

In [ ]:
model_df['Cluster_PCA'] = X_principal['Cluster'].values
model_df.head()

In [ ]:
sns.countplot(x = model_df['Cluster_PCA'])

In [ ]:
for num in range(7):
  cluster_length = len(model_df[model_df['Cluster_PCA'] == num])
  print('Length of Cluster {l}:'.format(l = num), cluster_length)

In [ ]:
def multivariate_clusteredPCA_num(df, x, y):
    plt.figure(figsize=(8, 5))
    scatter = sns.scatterplot(
        x=x,
        y=y,
        hue='Cluster_PCA',
        size=1,
        sizes=[10],
        data=df,
        palette='Spectral'
    )
    handles, labels = scatter.get_legend_handles_labels()

    plt.legend(handles=handles[:-1], title='Cluster', loc='upper left', bbox_to_anchor=(1, 1), scatterpoints=1)
    plt.title('{y_value} by {c_value} vs {x_value}'.format(y_value=y, x_value=x, c_value='Cluster'))
    plt.show()

In [ ]:
no_outliers = remove_outliers(model_df, 'AMOUNT_deb')
multivariate_clusteredPCA_num(no_outliers, 'AGE', 'AMOUNT_deb')

In [ ]:
multivariate_clusteredPCA_num(model_df, 'AGE', 'Recency_deb')

In [ ]:
no_outliers = remove_outliers(model_df, 'Frequency_deb')
multivariate_clusteredPCA_num(no_outliers, 'AGE', 'Frequency_deb')

In [ ]:
incomes = model_df.columns[5:16]
for i in incomes:
  no_outliers = remove_outliers(model_df, 'AMOUNT_deb')
  clusterboxes(i, "AMOUNT_deb", "Cluster_PCA", no_outliers)

In [ ]:
no_outliers = remove_outliers(model_df, 'AMOUNT_deb')
clusterboxes("EDUCATION", "AMOUNT_deb", "Cluster_PCA", no_outliers)

In [ ]:
incomes = model_df.columns[5:16]
for i in incomes:
  sns.countplot(data=model_df, x=i, hue='Cluster_PCA')
  plt.title('{income} by Cluster'.format(income=i))
  plt.show()

In [ ]:
sns.countplot(data=model_df, x='EDUCATION', hue='Cluster_PCA')
plt.title('EDUCATION by Cluster')
plt.show()

The following are the characteristics of each cluster (number at the end is the rank in terms of quantity):
*   **Cluster 0**: Old, high spender, remittance people, mostly mid educ, 5
*   **Cluster 1**: Middle-aged, low spender, salaried, mostly mid educ, low frequency, 2
*   **Cluster 2**: Scattered, mostly mid educ, 7
*   **Cluster 3**: Old, low spender, pension, mostly low educ, 4
*   **Cluster 4**: Young, mostly low educ, allowance 3
*   **Cluster 5**: Middle-aged, mid-high spender, salaried (almost all), mostly mid educ, mid-high frequency, 1
*   **Cluster 6**: Scattered, business people, mostly mid educ, 6



Cluster characteristics are identical with that of KMeans without PCA. But the clustering here is better as there is a higher silhouette score.

## DBSCAN

We attempted to employ DBSCAN on the dataset in order to manipulate the epsilon and min_samples parameters instead of manually finding the optimal number of clusters. This was done to check if the dataset can be segmented according to density.

In [ ]:
def multivariate_label_num(df, x, y):
    plt.figure(figsize=(8, 5))
    scatter = sns.scatterplot(
        x=x,
        y=y,
        hue='labels_dbscan',
        size=1,
        sizes=[10],
        data=df,
        palette='Spectral'
    )
    handles, labels = scatter.get_legend_handles_labels()

    plt.legend(handles=handles[:-1], title='Cluster', loc='upper left', bbox_to_anchor=(1, 1), scatterpoints=1)
    plt.title('{y_value} by {c_value} vs {x_value}'.format(y_value=y, x_value=x, c_value='labels_dbscan'))
    plt.show()

In [ ]:
db_model = model_df_scaled[model_df_scaled.columns.tolist()[:-1]]
db_model.head()

In [ ]:
eps_values = [0.5, 1, 2, 3, 4]
min_samples = [5, 50, 100, 200, 500]

In [ ]:
for e in eps_values:
  for samp in min_samples:
    db = DBSCAN(eps=e, min_samples = samp)
    labels = db.fit_predict(db_model)
    model_df['labels_dbscan'] = labels
    no_outliers = remove_outliers(model_df, 'AMOUNT_deb')
    print('Epsilon: {ep} Min_Samples: {sa}'.format(ep=e, sa=samp))
    multivariate_label_num(no_outliers, 'AGE', 'AMOUNT_deb')

In [ ]:
db = DBSCAN(eps=2, min_samples = 500)
labels = db.fit_predict(db_model)
model_df['labels_dbscan'] = labels
no_outliers = remove_outliers(model_df, 'AMOUNT_deb')
print('Epsilon: {ep} Min_Samples: {sa}'.format(ep=2, sa=500))
multivariate_label_num(no_outliers, 'AGE', 'AMOUNT_deb')

Epsilon value of 2 with Min_Samples equal to 500 seem to be an optimal value. Higher epsilon values yield uninterpretable clusters. High min_samples are needed to lessen cluster. The segments created are almost the same as the results of KMeans but there are obvious overlapping of clusters. The clustering does not also appear to divide according to the transaction amounts. This may be due to the fact that the data points are placed in such a way that it forms one dense cluster.

#Conclusion from Clustering

The clustering technique that yielded the best result was KMeans with PCA. It had a higher silhouette score than KMeans without PCA, even if the characteristics of each cluster were similar. The best clustering result in DBSCAN had somewhat similar clustering, but there were obvious overlapping of clusters.

# Additional Data Science/Data Analytics Methods

## Additional Cluster Validation Techniques

Silhouette Scores, Davies-Bouldin Indices, and Calinski-Harabasz Indices per Clustering Algorithms are added in order to validate our choice for KMeans with PCA. Higher silhouette and Calinski-Harabasz Index indicate good clustering and separation, while lower Davies-Bouldin Index indicates better clustering.

Source: https://www.linkedin.com/advice/0/what-some-alternative-metrics-evaluate-k-means

In [ ]:
sil_metrics = [silhouette_score(X_principal.iloc[:, :-1], model_df['Cluster_PCA']),
                  silhouette_score(model_df_scaled.iloc[:, :-1], model_df['Cluster']),
                  silhouette_score(db_model, model_df['labels_dbscan'])]
plt.bar(['KMeans with PCA', 'KMeans without PCA', 'DBSCAN'], sil_metrics)
plt.title('Silhouette Score per Clustering Algorithms')
plt.xlabel('Clustering Algorithm')
plt.ylabel('Silhouette Score')
plt.show()

In [ ]:
dav_metrics = [davies_bouldin_score(X_principal.iloc[:, :-1], model_df['Cluster_PCA']),
                  davies_bouldin_score(model_df_scaled.iloc[:, :-1], model_df['Cluster']),
                  davies_bouldin_score(db_model, model_df['labels_dbscan'])]
plt.bar(['KMeans with PCA', 'KMeans without PCA', 'DBSCAN'], dav_metrics)
plt.title('Davies-Bouldin Index per Clustering Algorithms')
plt.xlabel('Clustering Algorithm')
plt.ylabel('Davies-Bouldin Index')
plt.show()

In [ ]:
cal_metrics = [calinski_harabasz_score(X_principal.iloc[:, :-1], model_df['Cluster_PCA']),
                  calinski_harabasz_score(model_df_scaled.iloc[:, :-1], model_df['Cluster']),
                  calinski_harabasz_score(db_model, model_df['labels_dbscan'])]
plt.bar(['KMeans with PCA', 'KMeans without PCA', 'DBSCAN'], cal_metrics)
plt.title('Calinski-Harabasz Index per Clustering Algorithm')
plt.xlabel('Clustering Algorithm')
plt.ylabel('Calinski-Harabasz Index')
plt.show()

KMeans with PCA, indeed, appears to be the best choice. Even if KMeans without PCA appears to have the lowest Davies-Bouldin Index, it is almost identical to KMeans with PCA, and KMeans with PCA dominate the other metrics significantly.

## Cluster 5 Data Exploration

### Add Cluster label in original merged dataframe

In [ ]:
merged_df.head()

In [ ]:
merged_df_deb = merged_df.dropna(subset=['AMOUNT_deb'])
merged_df_deb.shape

In [ ]:
model_df.head()

In [ ]:
model_df.shape

In [ ]:
merged_df_deb_cluster = merged_df_deb.merge(model_df[['Cluster_PCA']], left_index=True, right_index=True, how='outer')

In [ ]:
merged_df_deb_cluster.head()

In [ ]:
sns.countplot(x = merged_df_deb_cluster['Cluster_PCA'])

In [ ]:
merged_df_deb_cluster.shape

In [ ]:
merged_df_deb_cluster.columns

### Cluster 5 only

In [ ]:
merged_df_deb_cluster5 = merged_df_deb_cluster[merged_df_deb_cluster['Cluster_PCA'] == 5]
merged_df_deb_cluster5.head()

In [ ]:
merged_df_deb_cluster5.shape

### EDA (no outliers)

In [ ]:
merged_df_deb_cluster5.describe(include='all')

In [ ]:
merged_df_deb_cluster5_cat_bar = ['PROVINCE', 'REGION', 'INCOME_SOURCE']
plt.figure(figsize=(10, 40))

for i, c in enumerate(merged_df_deb_cluster5_cat_bar):
  plt.subplot(3, 1, i+1)
  merged_df_deb_cluster5[c].value_counts().sort_values(ascending=True).plot(
      kind='barh',
      title=f"Bar Graph of Frequency of {c}",
      ylabel=c,
      xlabel='Frequency',
      fontsize=8
  )

In [ ]:
merged_df_deb_cluster5_cat_pie = ['EDUCATION', 'MARITAL_STATUS', 'BUSINESS_OWNER', 'DIGITAL_FLAG','GENDER']

plt.figure(figsize=(20, 20))

for i, c in enumerate(merged_df_deb_cluster5_cat_pie):
  cat = c.title().replace('_', ' ')
  plt.subplot(5, 3, i+1)
  plt.style.use('fast')

  sorted = merged_df_deb_cluster5[c].value_counts().sort_values(ascending=False)
  sorted.plot(kind='pie', fontsize=9, autopct='%1.f%%', startangle=-270, labels=None, counterclock=False)
  plt.legend(labels=sorted.rename(lambda x: x.title().replace('_', ' ')).index, bbox_to_anchor=(1, 1))
  plt.title(f"Proportion of Customers' {cat + 'ship' if c == 'BUSINESS_OWNER' else cat}", weight="bold")
  plt.tight_layout()
  plt.ylabel('')

There appears to be noticeable unevenness in Gender. There are considerably more Females in Cluster 5.

In [ ]:
merged_df_deb_cluster5_cat_hist = {'AGE':0, 'TENURE':1}
plt.figure(figsize=(30, 30))
for i,c in enumerate(merged_df_deb_cluster5_cat_hist):
  plt.subplot(4,3, i+1)
  n = remove_outliers(merged_df_deb_cluster5, c)
  n[c].plot(kind='hist', title=f"Histogram of {c}", xlabel=c)

Age Bracket 25-30 is most frequent.

In [ ]:
cred_no_outliers = remove_outliers(merged_df_deb_cluster5, 'AMOUNT_cred')
cred_no_outliers['AMOUNT_cred'].plot(kind='hist', title="Histogram of AMOUNT_cred")
plt.show()

In [ ]:
deb_no_outliers = remove_outliers(merged_df_deb_cluster5, 'AMOUNT_deb')
deb_no_outliers['AMOUNT_deb'].plot(kind='hist', title="Histogram of AMOUNT_deb")
plt.show()

Total Debit Card Transactions of ~P20K to ~P40K is most frequent.

In [ ]:
fin_no_outliers = remove_outliers(merged_df_deb_cluster5, 'AMOUNT_fin')
fin_no_outliers['AMOUNT_fin'].plot(kind='hist', title="Histogram of AMOUNT_fin")
plt.show()

In [ ]:
inc_inst_no_outliers = remove_outliers(merged_df_deb_cluster5, 'AMOUNT_inc_inst')
inc_inst_no_outliers['AMOUNT_inc_inst'].plot(kind='hist', title="Histogram of AMOUNT_inc_inst")
plt.show()

In [ ]:
out_inst_no_outliers = remove_outliers(merged_df_deb_cluster5, 'AMOUNT_out_inst')
out_inst_no_outliers['AMOUNT_out_inst'].plot(kind='hist', title="Histogram of AMOUNT_out_inst")
plt.show()